In [ ]:
from os import listdir
from os.path import isfile, join
import random 
import shutil

In [ ]:
SEED = 33
LABEL_PATH = 'region_data/region_labels'
IMAGE_PATH = 'region_data/region_images'

In [ ]:

def train_test_split(train_size, valid_size, random_state = None, shuffle = True):
    random.seed(random_state)
    image_files = [join(IMAGE_PATH, f) for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]
    if shuffle: random.shuffle(image_files)
    
    train_file = open("region_data/train.txt", "w+")
    valid_file = open("region_data/valid.txt", "w+")
    test_file = open("region_data/test.txt", "w+")
    train_number,valid_number,test_number = 0, 0, 0
    
    for f in image_files:
        u = random.uniform(0, 1)
        f += '\n'
        if u < train_size:
            train_file.write(f)
            train_number += 1
            continue
        elif u < train_size + valid_size:
            valid_file.write(f)
            valid_number += 1
            continue
        test_file.write(f)
        test_number += 1
    print("train {}, valid {}, test {}".format(train_number, valid_number, test_number))
    train_file.close()
    valid_file.close()
    test_file.close()

In [ ]:
train_test_split(0.7, 0.1, random_state = SEED)

In [ ]:
#!pip install -r requirements.txt

In [ ]:
!python3 train.py --epochs 1 \
                  --n_cpu 0 --img_size  416 \
                  --pretrained_weights checkpoints/yolov3_ckpt_0.pth \
                  --model_def config/yolov3-region.cfg \
                  --data_config config/region.data \
                  --batch_size 2 \
                  --evaluation_interval 10 \
                  --multiscale_training False

In [ ]:
import torch
torch.cuda.is_available() 

In [ ]:
print(torch.version.cuda)

In [ ]:
!python3 test.py  --n_cpu 0 \
                  --model_def config/yolov3-region.cfg \
                  --data_config config/region.data \
                  --class_path data/region.names \
                  --batch_size 2 \
                  --weights_path checkpoints/yolov3_ckpt_0.pth 

In [ ]:
!python3 detect.py --image_folder region_data/region_test_image \
                   --model_def config/yolov3-region.cfg \
                   --class_path region_data/region.names \
                   --weights_path checkpoints/yolov3_ckpt_0.pth \
                   --nms_thres 0.01

In [ ]:
# copy test image in data/test.txt to folder data/test_image


for image_path in open('region_data/region_test.txt', "r").readlines():
    shutil.copy(image_path.rstrip(), "region_data/region_test_image")